In [1]:
import codecs
import glob
import math
import os
from decimal import Decimal
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
data_dir = "data/input/励起状態/beta/NH/*.out"
paths = [p for p in glob.glob(data_dir)]

In [3]:
# とりあえずfを取り出して見る。
# key
excited_state_1 = "Excited State   1"
excited_state_2 = "Excited State   2"
excited_state_3 = "Excited State   3"

initial_key = "Excited State   1"
end_key = "SavETr"
result_dir = "data/output/result.csv"
df_output = pd.DataFrame()

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        f_1 = df[df['Col_1'].str.contains(excited_state_1)]['Col_1'].iloc[0].split(' ')
        f_1 = [a for a in f_1 if a != '']
        f_1 = float(f_1[-2].split('f=')[-1])
        
        f_2 = df[df['Col_1'].str.contains(excited_state_2)]['Col_1'].iloc[0].split(' ')
        f_2 = [a for a in f_2 if a != '']
        f_2 = float(f_2[-2].split('f=')[-1])
        
        f_3 = df[df['Col_1'].str.contains(excited_state_3)]['Col_1'].iloc[0].split(' ')
        f_3 = [a for a in f_3 if a != '']
        f_3 = float(f_3[-2].split('f=')[-1])
        
        df_f = pd.DataFrame(data=[f_1, f_2, f_3], 
                            columns=[torsion_angle],
                            index=["f_1", "f_2", "f_3"]).T
        df_output = pd.concat([df_output, df_f])
         

In [4]:
df_output = df_output.sort_index()